In [116]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import f1_score
import os
import cv2
import yaml
from ultralytics import YOLO
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Dropout,Layer,Conv2D,InputLayer,MaxPool2D,BatchNormalization,Input
from tensorflow.keras.metrics import BinaryAccuracy,FalsePositives,FalseNegatives,TruePositives,TrueNegatives,Precision,Recall, AUC
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import  Callback,CSVLogger,EarlyStopping,LearningRateScheduler
from sklearn.metrics import confusion_matrix,roc_curve
import pydot
import graphviz
import seaborn as sns
import matplotlib.pyplot as plt


In [117]:

train_dir = 'D:\juvdv2-vdvwc\Train\images'
val_dir = 'D:\juvdv2-vdvwc\Val\images'
train_annotations_dir = 'D:\juvdv2-vdvwc\Train\labels'
val_annotations_dir = 'D:\juvdv2-vdvwc\Val\labels'

In [243]:
def parse_annotation(annotation_dir):
    annotations = []
    for weather in ['Rainny', 'Sunny']:
        for time in ['Day', 'Night']:
            current_annotation_dir = os.path.join(annotation_dir, weather, time)
            for filename in os.listdir(current_annotation_dir):
                if filename.endswith(".txt"):
                    with open(os.path.join(current_annotation_dir, filename)) as f:
                        lines = f.readlines()
                        for line in lines:
                            label = line.split(' ')[0]
                            annotations.append(label)
    return annotations

In [213]:
def load_data(image_dir, annotation_dir):
    images = []
    labels = []
    label_counts = {}

    for weather in ['Rainny', 'Sunny']:
        for time in ['Day', 'Night']:
            curr_image_dir = os.path.join(image_dir, weather, time)
            curr_annotation_dir = os.path.join(annotation_dir, weather, time)
            
            for filename in os.listdir(curr_image_dir):
                if filename.endswith(".jpg") or filename.endswith(".jpeg"):
                    image = cv2.imread(os.path.join(curr_image_dir, filename))
                    image = cv2.resize(image, (224, 224))
                    images.append(image)
                    
                    annotation_file = os.path.join(curr_annotation_dir, filename.split('.')[0] + '.txt')
                    if os.path.exists(annotation_file):
                        with open(annotation_file) as f:
                            lines = f.readlines()
                            image_labels = []
                            
                            for line in lines:
                                label = line.split(' ')[0]
                                image_labels.append(label)
                                
                                if label not in label_counts:
                                    label_counts[label] = 1
                                else:
                                    label_counts[label] += 1
                            
                            labels.append(image_labels if image_labels else [])  # Append empty list if no labels
                    else:
                        labels.append([])  # No labels for this image

    return np.array(images), np.array(labels), label_counts


In [238]:
x_train,y_train,train_label_counts = load_data(train_dir,train_annotations_dir)
x_val,y_val,val_label_counts = load_data(val_dir,val_annotations_dir)

In [240]:
print('Number of images loaded:', len(x_train))
print('Number of labels loaded:', len(y_train))

Number of images loaded: 2600
Number of labels loaded: 17099


In [188]:
x_train.shape

(2600, 224, 224, 3)

In [190]:
len(y_train)

2600

In [192]:
x_train = x_train / 255.0
x_val = x_val / 255.0

In [193]:
train_annotations = parse_annotation(train_annotations_dir)
val_annotations = parse_annotation(val_annotations_dir)

In [194]:

print('Train label counts:',train_label_counts)

Train label counts: {'0': 10146, '1': 2379, '13': 166, '2': 1020, '3': 1194, '10': 9, '4': 291, '6': 331, '8': 333, '7': 343, '5': 387, '9': 315, '12': 2, '14': 49, '11': 132, '\n': 1, '0.679479': 1}


In [195]:
class customDense(Layer):
    def __init__(self, output_units, activation=None):
        super(customDense, self).__init__()
        self.output_units = output_units  
        self.activation = activation

    def build(self, input_features_shape):
        self.w = self.add_weight(shape=(input_features_shape[-1], self.output_units),
                                 initializer="random_normal",
                                 trainable=True)  
        
        self.b = self.add_weight(shape=(self.output_units,),
                                 initializer="random_normal",
                                 trainable=True)  

    def call(self, input_features):
        pre_output = tf.matmul(input_features, self.w) + self.b  
        if self.activation == "relu":
            return tf.nn.relu(pre_output)  
        elif self.activation == "sigmoid":
            return tf.math.sigmoid(pre_output)  
        else:
            return pre_output   


In [196]:

IM_SIZE =224
lenet_custom_model = tf.keras.Sequential([
    InputLayer(input_shape = (IM_SIZE,IM_SIZE,3)),
    Conv2D(
        filters = 6,
        kernel_size = 5,
        strides=(1,1),
        padding ='valid',
        activation = 'relu'  
    ),
    BatchNormalization(),
    MaxPool2D(
        pool_size = 2,
        strides = 2
    ),

    Conv2D(
        filters = 16,
        kernel_size = 5,
        strides=(1,1),
        padding ='valid',
        activation = 'relu'  
    ),
    BatchNormalization(),
    MaxPool2D(
        pool_size = 2,
        strides = 2
    ),

    Flatten(),

    customDense(100,activation="relu"),
    BatchNormalization(),
    
    customDense(10,activation="relu"), 
    BatchNormalization(),
    customDense(1,activation="sigmoid"),
])

c:\Users\chaga\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [197]:
lenet_custom_model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 220, 220, 6)    │           456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_15          │ (None, 220, 220, 6)    │            24 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 110, 110, 6)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 106, 106, 16)   │         2,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_16          │ (None, 106, 106, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 53, 53, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 44944)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ custom_dense_9 (customDense)    │ (None, 100)            │     4,494,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_17          │ (None, 100)            │           400 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ custom_dense_10 (customDense)   │ (None, 10)             │         1,010 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_18          │ (None, 10)             │            40 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ custom_dense_11 (customDense)   │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,498,921 (17.16 MB)

 Trainable params: 4,498,657 (17.16 MB)

 Non-trainable params: 264 (1.03 KB)

In [198]:
metrics = [TruePositives(name ='tp'),TrueNegatives(name='tn'),FalsePositives(name='fp'),FalseNegatives(name='fn'),
           Precision(name='precision'),Recall(name='recall'),AUC(name='auc'),BinaryAccuracy(name='accuracy')]

In [199]:
def scheduler(epoch,lr):
    if epoch<=10:
        return lr
    else:
        return (lr*tf.math.exp(-0.1)).numpy()
scheduler_callback = LearningRateScheduler(scheduler,verbose=1)

In [200]:
lenet_custom_model.compile(
    optimizer=Adam(learning_rate = 0.01),
    loss = BinaryCrossentropy(),
    metrics = metrics
)

In [245]:
history = lenet_custom_model.fit( x_train, y_train, batch_size=32, epochs=50, validation_data=(x_val, y_val),callbacks=[scheduler_callback])

ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 2600
'y' sizes: 17099
